# Process Raw Eye Movement Script (Eye Movements Derived Measures)

### Imports + Global Setup

In [1]:
from __future__ import print_function, division
%matplotlib inline

import sys 

def add_module_path(module_path):
    if module_path not in sys.path:
        sys.path.append(module_path)

add_module_path("../colour/")
add_module_path("../PyEDF/")

import numpy as np
import pandas as pd
import re
import os
import warnings
import math

from matplotlib import pyplot as plt
import scikits.bootstrap as boot

from operator import itemgetter

# import pyedf
# import conf

pd.options.display.mpl_style = 'default'

plt.rcParams['figure.figsize'] = (16.0, 10.0)
plt.set_cmap('gnuplot2')



C:\Users\xz32\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2881: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


## Global Constants

In [2]:
# path = './DataDummy'
#path = r'G:\Miguel\Research\Research-Current\CMOS\Git Experiment Only\Logs (For test analysis)'
#path = r'G:\Miguel\Research\Research-Current\CMOS\pilot data'
path = r'.\rawData'
#participant_ids = [ 'None', 'Johanas' ]
participant_ids = [  '15FM24',
                     '21MDC20',
                     '3MW2421',
                     '22F',
                     '14JG23',
                     '1MQ2201',
                     '2ML3603',
                     '8FC02',
                     '5MS3205',
                     '9FH04',
                     '1MW2207',
                     '12FRH06',
                     '6MDH09',
                     '13MM08',
                     '12JR11',
                     '10F',
                     '17MRW13',
                     '12F',
                     'MPS15',
                     '2FQ2014',
                     '5MS17',
                     '2FS2116',
                     '18MDM19',
                     '18F']
condition_ids = ['e', 'c', 's', 'n']
#condition_ids = ['e', 'c']
mediaObjectsPerDocument = 20 # this comes from how the documents were set up.
mediaObjectsScotland = 4 # this comes from how the documents were set up.

## Utility Functions 

In [3]:
def loadFileOfParticipantConditionLogtype(path,participantID,condition,logTypeEnding):
# Selects and loads a file from a participantID, of a given condition, of a particular type    
    regular_expression = str(participantID)+"__\S+_"+condition+"\S*"+logTypeEnding+".csv"
    filere = re.compile(regular_expression)
    frames = []
    numProcessedFiles = 0;    
    print('Loading file of type: '+logTypeEnding+", condition: "+condition+" , participant: "+participantID)
    for filename in os.listdir(path):      
        filematch = filere.match(filename)
        if filematch:
            filepath = path+'/'+filename
            print(filepath)            
            frame = pd.read_csv(filepath,delimiter=',',engine="python",header=0)
            print("Rows loaded: "+str(len(frame.index)))  
            frames.append(frame)            
            numProcessedFiles += 1;
        #else:
            #print("Tried to match: "+filename)
            #print("With RE: "+regular_expression)
    print(str(numProcessedFiles)+" processed files.")
    if (numProcessedFiles > 2):
        warnings.warn("Warning!!: I'm getting more than two files for the same participant and the same condition. Clash in ID's?")
    elif (numProcessedFiles < 2):
        warnings.warn("Warning!!: I should be finding two files for participant:"+participantID+" and condition:"+condition+" and I'm getting less.")
    return frames

def cleanSpacesInColumns(frame):
    columns = frame.columns
    newColumns = []
    ix = 0
    for columnName in columns:
        newColumns.append(columnName.strip(' \n\t'))
        ix += 1
    frame.columns = newColumns
    return frame

# From: http://stackoverflow.com/questions/6987285/python-find-the-item-with-maximum-occurrences
# Ned Deili
def max_occurrences(seq):
    c = dict()
    for item in seq:
        c[item] = c.get(item, 0) + 1
    return max(c.iteritems(), key=itemgetter(1))

## Analysis Functions

In [4]:
def processRawFixations(rawFixationsFrame):
        
    # first clean up unnecessary columns, and rename the x
    frame = cleanSpacesInColumns(rawFixationsFrame)
        # renaming long name
    mycolumns = list(frame.columns)
    ix = mycolumns.index('events(start fixation = 1 start saccade = 2)')
    mycolumns[ix] = 'events'
    frame.columns = mycolumns
    
    # Prepare the output of this function: it's a dictionary
    values = {}
    valueMeanings = {}

    values['userId'] = frame.iloc[0].userId
    valueMeanings['userId'] = 'the identification number of the user'
    values['type'] = frame.iloc[0].type
    valueMeanings['type'] = 'the type of trial, i.e., the condition. One of (e=eyecantext, c=cantext, s=static, n=no-images)'
    values['country'] = frame.iloc[0].country
    valueMeanings['country'] = 'the document the trial was done in. One of (0=Scotland(training), 1=Laos, 2=Vanuatu, 3=Suriname, 4=Equatorial Guinea)'
    values['orderNumber'] = frame.iloc[0].orderNumber
    valueMeanings['orderNumber'] = 'the id of the order in which participants faced the conditions and the documents. Goes from 0 to 23'
    values['experimentNumber'] = frame.iloc[0].experimentNumber
    valueMeanings['experimentNumber'] = 'to be completed'
    values['training'] = frame.iloc[0].training
    valueMeanings['experimentNumber'] = 'whether this data comes from training or not'
    
        
    # Cleanup: Determine where to start and end
        # Find first line that indicates started fixation that is in text or media object
    firstRow = frame[(frame.events == 1) & (frame.contentType != 'none')].iloc[0]
    firstTimeStamp = firstRow['timestamp']
    #print('first value:'+str(firstTimeStamp))
        # Find last line that is in text or media object
    lastRow = frame[(frame.events == 2) & (frame.contentType != 'none')].iloc[-1]
    lastTimeStamp = lastRow['timestamp']
    #print('last value:'+str(lastTimeStamp))
    
        # Actual cleanup (cut values above first and below last)
    frame = frame[(frame.timestamp >= firstTimeStamp) & (frame.timestamp <= lastTimeStamp)]
    #print("new length! "+str(len(frame.index)))     
  
        # Calculate duration of trial (store M10)
    values['M11'] = (lastTimeStamp - firstTimeStamp)/float(1000)
    valueMeanings['M11'] = 'The duration of a trial in seconds'        
    
    # Create a list of fixations, with type, object, duration    
    fixationStartsAndEnds = frame[(frame.events != 0)]
    
    # Trim the list of fixations, so that it always starts with a fixation
    firstFixation = fixationStartsAndEnds[(fixationStartsAndEnds.events == 1)].iloc[0]
    firstFixationTimeStamp = firstFixation['timestamp']
    fixationStartsAndEnds = fixationStartsAndEnds[(fixationStartsAndEnds.timestamp >= firstFixationTimeStamp)]
    #print('first fixation is now: '+str(fixationStartsAndEnds['events'].iloc[0]))
    #print('the one after is now : '+str(fixationStartsAndEnds['events'].iloc[1]))
    
    fixations = pd.DataFrame(columns=['start','end','duration','type','object','avgX','avgY'])
    saccades = pd.DataFrame(columns=['start','end','duration','distance','fromType','toType','fromObject','toObject','destX','destY'])
    
    ixFSE = 0 # index to iterate over fixationsStartsAndEnds
    ixF = 0 # index to iterate over fixations
    ixS = -1 # index to iterate over sacaddes
    fixationCounter = 0
    
    # Variables hold necessary previous data for the calculation of saccades between fixations
    startSaccadeX = -1; 
    startSaccadeY = -1;
    startSaccade = -1;
    startSaccadeType = "Error";
    startSaccadeObject = "Error";
                            
    while True: # processing the pairs of events

        if (not(len(fixationStartsAndEnds)>ixFSE+1)):
            break # break when we are done processing the length of fixationStartsAndEnds

        while (fixationStartsAndEnds.iloc[ixFSE]['events'] == 2): # if the fixation start is not a Start (1) then there is something wrong
            print('Fixation and Saccade segmentation started with a saccade. ixFSE = '+str(ixFSE))
            print('Moving on by one')
            ixFSE += 1
            
        # ************ First we deal with the fixations
            
        # calculating the duration of this fixation
        start = fixationStartsAndEnds.iloc[ixFSE].timestamp
        
        while (fixationStartsAndEnds.iloc[ixFSE+1].events == 1): # if this is still a fixation then we have to group them
            ixFSE += 1
        end = fixationStartsAndEnds.iloc[ixFSE+1].timestamp
        duration = end-start
                            
        # calculating the content type of this fixation
        fixationSubset = frame[(frame.timestamp >= start) & (frame.timestamp < end)]
        mostCommonContentType = max_occurrences(list(fixationSubset['contentType']))
        mostCommonContent = max_occurrences(list(fixationSubset['content']))
        
        #calculating the average position of this fixation
        avgX = np.sum(fixationSubset['x'])/float(len(list(fixationSubset['x'])))
        avgY = np.sum(fixationSubset['y'])/float(len(list(fixationSubset['y'])))
        
        if(math.isnan(avgX) or math.isnan(avgY)):
            print("Found a problem in the calculation of the position of a fixation")
            #print("sumX = "+str(np.sum(list(fixationSubset['x']))))
            #print(fixationSubset['x'])
            #print("sumY = "+str(np.sum(list(fixationSubset['x']))))
            #print(fixationSubset['y'])
            #print("Length of fixation subset: "+str(len(list(fixationSubset['x']))))
        
        
        fixations.loc[ixF] = {'start': start,'end': end,'duration': duration,'type': mostCommonContentType[0],'object': mostCommonContent[0],'avgX': avgX,'avgY': avgY}

        # ************ Now we deal with the saccades
        
        while ( (len(fixationStartsAndEnds) > (ixFSE+2)) and (fixationStartsAndEnds.iloc[(ixFSE+2)].events == 2)): # for when there are two consecutive saccades we advance one
            print("Two saccades after each other in ixFSE: "+str(ixFSE))
            ixFSE +=1
            
        
        if (ixS >= 0): # there can only be a saccade after there has been a fixation
        # calculating duration of this saccade
        
            saccadeEnd = start # the end of the previous saccade is the start of this fixation
            saccadeDuration = saccadeEnd - startSaccade
            saccadeDistance = np.sqrt((avgX-startSaccadeX)**2 +(avgY-startSaccadeY)**2)
            
            if (math.isnan(saccadeDistance)):
                print("avgX:"+str(avgX)+" avgY: "+str(avgY)+" startSaccadeX: "+str(startSaccadeX)+" startSaccadeY: "+str(startSaccadeY))
                print("Distance is NaN!!: ixFSE: "+str(ixFSE))
            
            saccadeToType = mostCommonContentType[0]
            saccadeToObject = mostCommonContent[0]
        
            saccades.loc[ixS] = {'start':startSaccade,'end':saccadeEnd,'duration':saccadeDuration,'distance':saccadeDistance,'fromType':startSaccadeType,'toType':saccadeToType,'fromObject':startSaccadeObject,'toObject':saccadeToObject,'destX':avgX,'destY':avgY}
            # print("saccade: "+str(saccades.loc[ixS]))
        
        
        # initialize data for next saccade
        startSaccadeX = avgX # assumes that the saccade goes from center of fixation to center of fixation
        startSaccadeY = avgY # ^^
        startSaccade = end
        startSaccadeType = mostCommonContentType[0]
        startSaccadeObject = mostCommonContent[0]
        
        
        # print('start:'+str(start)+" end:"+str(end)+" duration:"+str(duration)+" contentType:"+mostCommonContentType[0]+" content:"+mostCommonContent[0]+" avgX,avgY:"+str(avgX)+","+str(avgY))
        ixFSE += 2
        ixF += 1
        ixS += 1
        
        
    # Now we process the created data of fixations and saccades, to generate the new measures
            
        # Store average duration of fixations (M3), number of fixations overall (M4), 
        # number of fixations per second of trial (M4perSec)
    numFixations = len(fixations)
    totalFixationDuration = sum(fixations['duration'])/float(1000)
    avgFixationDuration = totalFixationDuration/float(numFixations)

    values['M12'] = totalFixationDuration
    valueMeanings['M12'] = 'The total time spent in fixations in the document'    
    values['M3'] = avgFixationDuration
    valueMeanings['M3'] = 'The average duration of fixations'
    values['M4'] = numFixations
    valueMeanings['M4'] = 'The number of fixations'
    values['M4perSec'] = numFixations/values['M11']
    valueMeanings['M4perSec'] = 'The number of fixations divided by the valid trial time'
                
        # Same as above but for text only 
        # (this could be done in a function to avoid code replication, but here legibility
        # by other researchers is more important)
        
    textFixations = fixations[(fixations['type']=='text')]    
    textNumFixations = len(textFixations)
    
    if (textNumFixations > 0):
        textTotalFixationDuration = sum(textFixations['duration'])/float(1000)
        textAvgFixationDuration = textTotalFixationDuration/float(textNumFixations)
    else:
        textTotalFixationDuration = 0
        textAvgFixationDuration = 0

    values['M12t'] = textTotalFixationDuration
    valueMeanings['M12t'] = 'The total time spent in fixations in text'    
    values['M3t'] = textAvgFixationDuration
    valueMeanings['M3t'] = 'The average duration of text fixations'
    values['M4t'] = textNumFixations
    valueMeanings['M4t'] = 'The number of text fixations'
    values['M13t'] = values['M12t']/float(values['M12'])
    valueMeanings['M13t'] = 'The proportion of fixation time on text'

        # Same as above but for media only 
        # (this could be done in a function to avoid code replication, but here legibility
        # by other researchers is more important)
        
    mediaFixations = fixations[(fixations['type']=='media')]    
    mediaNumFixations = len(mediaFixations)
    
    if (mediaNumFixations > 0):
        mediaTotalFixationDuration = sum(mediaFixations['duration'])/float(1000)
        mediaAvgFixationDuration = mediaTotalFixationDuration/float(mediaNumFixations)
    else:
        mediaTotalFixationDuration = 0
        mediaAvgFixationDuration = 0

    values['M12m'] = mediaTotalFixationDuration
    valueMeanings['M12m'] = 'The total time spent in fixations in media'    
    values['M3m'] = mediaAvgFixationDuration
    valueMeanings['M3m'] = 'The average duration of media fixations'
    values['M4m'] = mediaNumFixations
    valueMeanings['M4m'] = 'The number of media fixations'
    values['M13m'] = values['M12m']/float(values['M12'])
    valueMeanings['M13m'] = 'The proportion of fixation time on media'

        
        # Same as above but for other only 
        # (this could be done in a function to avoid code replication, but here legibility
        # by other researchers is more important)
        
    otherFixations = fixations[(fixations['type']=='none')]    
    otherNumFixations = len(otherFixations)
    
    if (otherNumFixations > 0):
        otherTotalFixationDuration = sum(otherFixations['duration'])/float(1000)
        otherAvgFixationDuration = otherTotalFixationDuration/float(otherNumFixations)
    else:
        otherTotalFixationDuration = 0
        otherAvgFixationDuration = 0

    values['M12o'] = otherTotalFixationDuration
    valueMeanings['M12o'] = 'The total time spent in fixations in other'    
    values['M3o'] = otherAvgFixationDuration
    valueMeanings['M3o'] = 'The average duration of other fixations'
    values['M4o'] = otherNumFixations
    valueMeanings['M4o'] = 'The number of other fixations' 
    values['M13o'] = values['M12o']/float(values['M12'])
    valueMeanings['M13o'] = 'The proportion of fixation time on other'

        
    # ************* Saccade stuff
        
        # average saccade distance (M5), average saccade duration (M6)
    numSaccades = len(saccades)
        
    avgSaccadeDistance = sum(saccades['distance'])/float(numSaccades)
    values['M5'] = avgSaccadeDistance
    valueMeanings['M5'] = 'The average saccade distance'    
    
    #print("sum Sacc Distance (all): "+str(sum(saccades['distance'])))
    #print("num Saccades      (all): "+str(numSaccades))
    if (math.isnan(sum(saccades['distance']))):
        print(saccades['distance'])
    
    saccadeDuration = sum(saccades['duration'])/float(1000)
    avgSaccadeDuration = saccadeDuration/float(numSaccades)
    values['M6'] = avgSaccadeDuration
    valueMeanings['M6'] = 'The average saccade duration'    
    
        # Same as above but for text-to-text saccades only 
        # (this could be done in a function to avoid code replication, but here legibility
        # by other researchers is more important)
    
    textSaccades = saccades[(saccades['fromType']=='text') & (saccades['toType']=='text')]
    
    textNumSaccades = len(textSaccades)
    if (textNumSaccades != 0):
        textAvgSaccadeDistance = sum(textSaccades['distance'])/float(textNumSaccades)
        textSaccadeDuration = sum(textSaccades['duration'])/float(1000)
        textAvgSaccadeDuration = textSaccadeDuration/float(textNumSaccades)
    else:
        textAvgSaccadeDistance = 0
        textSaccadeDuration = 0
        textAvgSaccadeDuration = 0
        
    values['M5t'] = textAvgSaccadeDistance
    valueMeanings['M5t'] = 'The average saccade distance for text-to-text saccades'            
    values['M6t'] = textAvgSaccadeDuration
    valueMeanings['M6t'] = 'The average saccade duration for text-to-text saccades'    
    
        # Same as above but for media-to-media saccades only 
        # (this could be done in a function to avoid code replication, but here legibility
        # by other researchers is more important)
    
    mediaSaccades = saccades[(saccades['fromType']=='media') & (saccades['toType']=='media')]
    
    mediaNumSaccades = len(mediaSaccades)
    if (mediaNumSaccades != 0):
        mediaAvgSaccadeDistance = sum(mediaSaccades['distance'])/float(mediaNumSaccades)
        mediaSaccadeDuration = sum(mediaSaccades['duration'])/float(1000)
        mediaAvgSaccadeDuration = mediaSaccadeDuration/float(mediaNumSaccades)
    else:
        mediaAvgSaccadeDistance = 0
        mediaSaccadeDuration = 0
        mediaAvgSaccadeDuration = 0
        
    values['M5m'] = mediaAvgSaccadeDistance
    valueMeanings['M5m'] = 'The average saccade distance for media-to-media saccades'            
    values['M6m'] = mediaAvgSaccadeDuration
    valueMeanings['M6m'] = 'The average saccade duration for media-to-media saccades'    

        # Same as above but for other-to-other saccades only 
        # (this could be done in a function to avoid code replication, but here legibility
        # by other researchers is more important)
    
    otherSaccades = saccades[(saccades['fromType']=='none') & (saccades['toType']=='none')]
    
    otherNumSaccades = len(otherSaccades)
    if (otherNumSaccades != 0):
        otherAvgSaccadeDistance = sum(otherSaccades['distance'])/float(otherNumSaccades)
        otherSaccadeDuration = sum(otherSaccades['duration'])/float(1000)
        otherAvgSaccadeDuration = otherSaccadeDuration/float(otherNumSaccades)
    else:
        otherAvgSaccadeDistance = 0
        otherSaccadeDuration = 0
        otherAvgSaccadeDuration = 0
        
    values['M5o'] = otherAvgSaccadeDistance
    valueMeanings['M5o'] = 'The average saccade distance for other-to-other saccades'            
    values['M6o'] = otherAvgSaccadeDuration
    valueMeanings['M6o'] = 'The average saccade duration for other-to-other saccades'

    # Create a list of entries and exits into media objects
    
    saccadesIntoMediaObjects = saccades[(saccades['toType']=='media')&(saccades['fromObject']!=saccades['toObject'])]
    values['M7'] = len(saccadesIntoMediaObjects)
    valueMeanings['M7']= 'The total number of entries into media items. Saccades within the same object discounted. Saccades from other media objects possible.'

    
    # Now we create a list of media objects that were ever fixated
    fixatedMediaList = fixations[(fixations['type']=='media')]['object']
    fixatedMediaSet = set(fixatedMediaList)
    values['M10'] = len(fixatedMediaSet)
    valueMeanings['M10'] = 'The number of media elements that were fixated at all during the trial'

    # Just for verification purposes (or future proofing, we also calculate the paragraphs and number of paragraphs fixated)
    fixatedTextList = fixations[(fixations['type']=='text')]['object']
    fixatedTextSet = set(fixatedTextList)
    values['V10t'] = len(fixatedTextSet)
    valueMeanings['V10t'] = 'The number of text elements (paragraphs) that were fixated at all during the trial'
    
    # Now we are going to count, for each object how many times it gets a saccade in
        # First we build a dictionary of objects
    numberOfEntriesInMediaPerObject = dict.fromkeys(fixatedMediaSet,0)
    for mediaObject in fixatedMediaSet:
        numEntries = len(saccadesIntoMediaObjects[(saccades['toObject']==mediaObject)])
        numberOfEntriesInMediaPerObject[mediaObject] = numEntries
    
    # Now we are going to count, for each object how much fixation time is spent in it
        # First we build a dictionary of objects
    fixationTimeInMediaPerObject = dict.fromkeys(fixatedMediaSet,0)    
    for mediaObject in fixatedMediaSet:
        fixationsOnMediaObject = fixations[(fixations['type']=='media')&(fixations['object']==mediaObject)]
        totalTime = sum(fixationsOnMediaObject['duration'])
        fixationTimeInMediaPerObject[mediaObject] = totalTime/float(1000)

    # For Verification purposes, we also calculate how long fixations take on each paragraph
        # First we build a dictionary of objects
    fixationTimeInTextPerObject = dict.fromkeys(fixatedTextSet,0)    
    for textObject in fixatedTextSet:
        fixationsOnTextObject = fixations[(fixations['type']=='text')&(fixations['object']==textObject)]
        totalTime = sum(fixationsOnTextObject['duration'])
        fixationTimeInTextPerObject[textObject] = totalTime/float(1000)

    #mediaObjectsPerDocument = 20 # this comes from how the documents were set up.
    #mediaObjectsScotland = 4 # this comes from how the documents were set up.
    if (values['country']==0):
        values['M8']=values['M7']/float(mediaObjectsScotland)
    else:
        values['M8']=values['M7']/float(mediaObjectsPerDocument)
    valueMeanings['M8']='The average number of entries into media items (M7) divided by the number of available media items (extracted by hand from each document)'

    #fixations.loc[ixF] = {'start': start,'end': end,'duration': duration,'type': mostCommonContentType[0],'object': mostCommonContent[0],'avgX': avgX,'avgY': avgY}
    #saccades.loc[ixS] = {'start':startSaccade,'end':saccadeEnd,'duration':saccadeDuration,'distance':saccadeDistance,'fromType':startSaccadeType,'toType':saccadeToType,'fromObject':startSaccadeObject,'toObject':saccadeToObject}

    # Calculate, for each entry on a media object, what is the distance from the last fixation
    # within text (if the previous fixation was not another media object)
    
    # saccadesIntoMediaObjects
    # saccades
    
    distances = []
    countValidSaccadesIntoMediaObjects = 0
    for ix in saccadesIntoMediaObjects.index:
        # get the timestamp of the sacade entry
        timestamp = saccadesIntoMediaObjects.loc[ix]['end']
        destinationX = saccadesIntoMediaObjects.loc[ix]['destX']
        destinationY = saccadesIntoMediaObjects.loc[ix]['destY']
        # get which object this saccade was into
        myObject = saccadesIntoMediaObjects.loc[ix]['toObject']
        # now find the last fixation that was not 'none'
        fixationsUntilSaccade = fixations[(fixations['end']<timestamp)&(fixations['type']!='none')&(fixations['object']!=myObject)]
        try:
            lastFixation = fixationsUntilSaccade.iloc[-1]
        except:
            warnings.warn("lastFix error")
            continue
        if (lastFixation['type']=='text'):
            positionFixationX = lastFixation['avgX']
            positionFixationY = lastFixation['avgY']
            distance = np.sqrt((destinationX-positionFixationX)**2+(destinationY-positionFixationY)**2)
            distances.append(distance)
            countValidSaccadesIntoMediaObjects += 1

    if (countValidSaccadesIntoMediaObjects>0):
        values['M9'] = sum(distances)/float(countValidSaccadesIntoMediaObjects)
    else:
        values['M9'] = float('NaN')
    valueMeanings['M9'] = 'Average distance (in pixels) of the last saccades in text that landed into a media object. Does not count saccades from other media objects or the object itself. If coming from a none, it finds the previous text fixation'
    #print('****************** last distance average:'+str(values['M9']))    
        
    return [values, valueMeanings]


## Main Analysis Below

In [5]:
# First Calculate Fixation and Saccade Metrics
# goes over every participant, condition, and repetition file and calculates the relevant metrics
fixationMeasuresDataRows = []
fixationMeasuresDataMeanings = []
rowCount = 0
for participantID in participant_ids:
    print('**-Participant:'+participantID)
    for conditionID in condition_ids:
        print('----Condition:'+conditionID)
        frames = loadFileOfParticipantConditionLogtype(path,
                                                           participantID,
                                                           conditionID,
                                                           'fixationData')
        len(frames)
        for frame in frames:
            processedDataRows = processRawFixations(frame)
            # print('len of processed data rows:'+str(len(processedDataRows)))
            fixationMeasuresDataRows.append(processedDataRows[0])
            rowCount += 1
            # print('rowCount:'+str(rowCount))
            if (len(fixationMeasuresDataMeanings) == 0):
                fixationMeasuresDataMeanings = processedDataRows[1]

fixationMeasures = pd.DataFrame(fixationMeasuresDataRows)
#fixationMeasuresDataMeanings = pd.DataFrame(fixationMeasuresDataMeanings)
fixationMeasures.to_pickle(r'.\processedData\fixationAnalysisResults_pandas.pddf')
fixationMeasures.to_csv(r'.\processedData\fixationAnalysisResults.csv')
#fixationMeasuresDataMeanings.to_csv('fixationAnalysisColumnExplanations.csv')

                    
    

**-Participant:15FM24
----Condition:e
Loading file of type: fixationData, condition: e , participant: 15FM24
.\rawData/15FM24__125_e_0_24__2016_9_16_16_33_8_fixationData.csv
Rows loaded: 4132
.\rawData/15FM24__126_e_3_24__2016_9_16_16_41_0_fixationData.csv
Rows loaded: 23356
2 processed files.
Two saccades after each other in ixFSE: 796
Two saccades after each other in ixFSE: 1780


C:\Users\xz32\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:344: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


----Condition:c
Loading file of type: fixationData, condition: c , participant: 15FM24
.\rawData/15FM24__123_c_0_24__2016_9_16_16_22_3_fixationData.csv
Rows loaded: 10329
.\rawData/15FM24__124_c_4_24__2016_9_16_16_30_41_fixationData.csv
Rows loaded: 23730
2 processed files.
Two saccades after each other in ixFSE: 864
Two saccades after each other in ixFSE: 1947
Two saccades after each other in ixFSE: 2018
Two saccades after each other in ixFSE: 2051
Two saccades after each other in ixFSE: 2133
Two saccades after each other in ixFSE: 2146
Two saccades after each other in ixFSE: 2157
----Condition:s
Loading file of type: fixationData, condition: s , participant: 15FM24
.\rawData/15FM24__129_s_0_24__2016_9_16_16_52_11_fixationData.csv
Rows loaded: 2107
.\rawData/15FM24__130_s_1_24__2016_9_16_17_0_40_fixationData.csv
Rows loaded: 26294
2 processed files.
----Condition:n
Loading file of type: fixationData, condition: n , participant: 15FM24
.\rawData/15FM24__127_n_0_24__2016_9_16_16_42_57_f

C:\Users\xz32\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:24: UserWarning: Warning!!: I should be finding two files for participant:1MQ2201 and condition:s and I'm getting less.


----Condition:n
Loading file of type: fixationData, condition: n , participant: 1MQ2201
.\rawData/1MQ2201__5_n_0_1__2016_1_22_16_17_18_fixationData.csv
Rows loaded: 1907
.\rawData/1MQ2201__6_n_4_1__2016_1_22_16_28_0_fixationData.csv
Rows loaded: 34864
2 processed files.
**-Participant:2ML3603
----Condition:e
Loading file of type: fixationData, condition: e , participant: 2ML3603
.\rawData/2ML3603__7_e_0_3__2016_4_19_16_21_38_fixationData.csv
Rows loaded: 5825
.\rawData/2ML3603__8_e_1_3__2016_4_19_16_30_26_fixationData.csv
Rows loaded: 17041
2 processed files.
----Condition:c
Loading file of type: fixationData, condition: c , participant: 2ML3603
.\rawData/2ML3603__11_c_0_3__2016_4_19_16_48_51_fixationData.csv
Rows loaded: 3706
.\rawData/2ML3603__12_c_2_3__2016_4_19_16_56_19_fixationData.csv
Rows loaded: 16206
2 processed files.
----Condition:s
Loading file of type: fixationData, condition: s , participant: 2ML3603
.\rawData/2ML3603__10_s_3_3__2016_4_19_16_45_42_fixationData.csv
Rows lo

C:\Users\xz32\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:24: UserWarning: Warning!!: I should be finding two files for participant:12JR11 and condition:e and I'm getting less.


----Condition:c
Loading file of type: fixationData, condition: c , participant: 12JR11
.\rawData/12JR11__105_c_0_11__2016_9_16_10_27_26_fixationData.csv
Rows loaded: 3188
.\rawData/12JR11__106_c_3_11__2016_9_16_10_37_6_fixationData.csv
Rows loaded: 36165
2 processed files.
----Condition:s
Loading file of type: fixationData, condition: s , participant: 12JR11
.\rawData/12JR11__103_s_0_11__2016_9_16_10_14_19_fixationData.csv
Rows loaded: 2732
.\rawData/12JR11__104_s_1_11__2016_9_16_10_25_33_fixationData.csv
Rows loaded: 35534
2 processed files.
----Condition:n
Loading file of type: fixationData, condition: n , participant: 12JR11
.\rawData/12JR11__100_n_0_11__2016_9_16_9_48_56_fixationData.csv
Rows loaded: 16070
.\rawData/12JR11__101_n_2_11__2016_9_16_9_58_30_fixationData.csv
Rows loaded: 35177
2 processed files.
**-Participant:10F
----Condition:e
Loading file of type: fixationData, condition: e , participant: 10F
.\rawData/10F__172_e_0_10__2016_12_13_15_56_30_fixationData.csv
Rows loade

C:\Users\xz32\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:394: UserWarning: lastFix error


----Condition:n
Loading file of type: fixationData, condition: n , participant: MPS15
.\rawData/MPS15__151_n_0_15__2016_9_19_13_56_22_fixationData.csv
Rows loaded: 1042
.\rawData/MPS15__152_n_1_15__2016_9_19_14_1_48_fixationData.csv
Rows loaded: 18768
2 processed files.
**-Participant:2FQ2014
----Condition:e
Loading file of type: fixationData, condition: e , participant: 2FQ2014
.\rawData/2FQ2014__32_e_0_14__2016_6_17_14_51_45_fixationData.csv
Rows loaded: 8798
.\rawData/2FQ2014__33_e_2_14__2016_6_17_15_0_13_fixationData.csv
Rows loaded: 30610
2 processed files.
----Condition:c
Loading file of type: fixationData, condition: c , participant: 2FQ2014
.\rawData/2FQ2014__28_c_0_14__2016_6_17_14_27_57_fixationData.csv
Rows loaded: 11090
.\rawData/2FQ2014__29_c_1_14__2016_6_17_14_35_57_fixationData.csv
Rows loaded: 30774
2 processed files.
----Condition:s
Loading file of type: fixationData, condition: s , participant: 2FQ2014
.\rawData/2FQ2014__30_s_0_14__2016_6_17_14_39_22_fixationData.csv


In [6]:
for row in fixationMeasures:
    print(row)
    
fixationMeasures

M10
M11
M12
M12m
M12o
M12t
M13m
M13o
M13t
M3
M3m
M3o
M3t
M4
M4m
M4o
M4perSec
M4t
M5
M5m
M5o
M5t
M6
M6m
M6o
M6t
M7
M8
M9
V10t
country
experimentNumber
orderNumber
training
type
userId


,M10,M11,M12,M12m,M12o,M12t,M13m,M13o,M13t,M3,...,M7,M8,M9,V10t,country,experimentNumber,orderNumber,training,type,userId
0,0,47.832,35.136,0.000,7.356,27.780,0.000000,0.209358,0.790642,0.276661,...,0,0.00,NaN,4,0,125,24,False,e,15FM24
1,12,280.572,217.356,15.936,22.044,179.376,0.073318,0.101419,0.825264,0.202568,...,21,1.05,541.613187,13,3,126,24,False,e,15FM24
2,5,120.252,91.824,1.908,14.700,75.216,0.020779,0.160089,0.819132,0.216056,...,8,2.00,627.670007,4,0,123,24,False,c,15FM24
3,13,286.536,220.632,9.408,22.944,188.280,0.042641,0.103992,0.853367,0.199667,...,22,1.10,575.071324,9,4,124,24,False,c,15FM24
4,7,23.652,18.996,6.660,7.968,4.368,0.350600,0.419457,0.229943,0.237450,...,13,3.25,523.865325,4,0,129,24,False,s,15FM24
5,19,327.636,245.184,19.116,39.060,187.008,0.077966,0.159309,0.762725,0.211366,...,44,2.20,367.932505,12,1,130,24,False,s,15FM24
6,0,17.436,14.208,0.000,1.452,12.756,0.000000,0.102196,0.897804,0.225524,...,0,0.00,NaN,4,0,127,24,False,n,15FM24
7,0,263.472,205.752,0.000,13.236,192.516,0.000000,0.064330,0.935670,0.208462,...,0,0.00,NaN,10,2,128,24,False,n,15FM24
8,2,14.364,11.496,0.804,1.584,9.108,0.069937,0.137787,0.792276,0.273714,...,3,0.75,516.187591,4,0,161,20,False,e,21MDC20
9,15,500.352,405.204,9.696,43.524,351.984,0.023929,0.107413,0.868659,0.240477,...,24,1.20,531.966727,10,2,162,20,False,e,21MDC20
